In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import random
import os

C:\Users\JangSeongHyun\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\JangSeongHyun\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\JangSeongHyun\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
    
seed_everything(42)

In [6]:
path = './beauty_3/'

In [7]:
df = pd.read_csv(path+'ratings_Beauty.csv')
df = df[['UserId', 'ProductId', 'Rating']]
df.head()

,UserId,ProductId,Rating
0,A39HTATAQ9V7YF,0205616461,5.0
1,A3JM6GV9MNOF9X,0558925278,3.0
2,A1Z513UWSAAO0F,0558925278,5.0
3,A1WMRR494NWEWV,0733001998,4.0
4,A3IAAVS479H7M7,0737104473,1.0


In [8]:
df.shape

(2023070, 3)

In [9]:
df.ProductId.nunique()

249274

## Preprocessing

## Dataset 생성

In [6]:
id_table = df['UserId'].value_counts().reset_index()
id_table.columns = ['UserId', 'count']
id_table['count'].value_counts().head(20)

1     887401
2     175875
3      64336
4      30285
5      16187
6       9827
7       6324
8       4260
9       3181
10      2275
11      1745
12      1402
13      1012
14       912
15       677
16       550
17       462
18       411
19       323
20       277
Name: count, dtype: int64

In [10]:
id_list = id_table.query('count == 3')['UserId'].tolist() # 5/10 4/6 3/4
data2 = df.query('UserId in @id_list')
data2.shape

(193008, 3)

In [11]:
len(id_list)

64336

In [9]:
for i in data2['UserId'].unique() : 
    in_lst = data2.loc[ data2['UserId'] == i ].index
    if len(in_lst) > 3 : 
        ne_in = np.random.choice(in_lst, 3, replace=False)
        de_in = list(set(in_lst) - set(ne_in))
        data2 = data2.drop(de_in, axis=0).reset_index(drop=True)
    else : 
        pass

In [12]:
import random

train_list = random.sample(id_list, 50000) # 30000 40000 50000
id_list = set(id_list) - set(train_list)
valid_list = random.sample(id_list, 7336) # 6054 8299 7336
id_list = set(id_list) - set(valid_list)
test_list = random.sample(id_list, 7000) # 6000 8000 7000

In [13]:
train = data2.query('UserId in @train_list').reset_index(drop=True)
valid = data2.query('UserId in @valid_list').reset_index(drop=True)
test = data2.query('UserId in @test_list').reset_index(drop=True)

train.shape, valid.shape, test.shape

((150000, 3), (22008, 3), (21000, 3))

In [16]:
train = train.sort_values(by='UserId')
valid = valid.sort_values(by='UserId')
test = test.sort_values(by='UserId')

In [17]:
# x_train = pad_sequences(train['encoded'], maxlen=maxlen, padding='post')
# x_valid = pad_sequences(valid['encoded'], maxlen=maxlen, padding='post')
# x_test = pad_sequences(test['encoded'], maxlen=maxlen, padding='post')

x_train = np.array(train['ProductId'])
x_valid = np.array(valid['ProductId'])
x_test = np.array(test['ProductId'])

In [18]:
description_num = 3

# Des_tr = x_train.reshape(25000, description_num, maxlen)
# Des_val = x_valid.reshape(2636, description_num, maxlen) # 2715 2425 2636
# Des_tst = x_test.reshape(2500, description_num, maxlen)

maxlen = 1
Des_tr = x_train.reshape(50000, description_num, maxlen)
Des_val = x_valid.reshape(7336, description_num, maxlen)
Des_tst = x_test.reshape(7000, description_num, maxlen)

In [19]:
Des = []

for i in range(Des_tr.shape[0]):
    for j in range(3):
        Des.append(Des_tr[i])
        
train_des = np.array(Des)

In [20]:
Des = []

for i in range(Des_val.shape[0]):
    for j in range(3):
        Des.append(Des_val[i])
        
valid_des = np.array(Des)

In [21]:
Des = []

for i in range(Des_tst.shape[0]):
    for j in range(3):
        Des.append(Des_tst[i])
        
test_des = np.array(Des)

In [22]:
train_data = [train_des, x_train]
valid_data = [valid_des, x_valid]
test_data = [test_des, x_test]

train_target = train[['Rating']].values
valid_target = valid[['Rating']].values
test_target = test[['Rating']].values

In [24]:
import pickle

with open(path+"Amazon_beauty/beauty_3/train_id_data.pickle","wb") as fr:
    pickle.dump(train_data, fr, protocol=4)
    
with open(path+"Amazon_beauty/beauty_3/valid_id_data.pickle","wb") as fr:
    pickle.dump(valid_data, fr, protocol=4)
    
with open(path+"Amazon_beauty/beauty_3/test_id_data.pickle","wb") as fr:
    pickle.dump(test_data, fr, protocol=4)    

In [25]:
with open(path+"Amazon_beauty/beauty_3/train_target.pickle","wb") as fr:
    pickle.dump(train_target, fr, protocol=4)
    
with open(path+"Amazon_beauty/beauty_3/valid_target.pickle","wb") as fr:
    pickle.dump(valid_target, fr, protocol=4)
    
with open(path+"Amazon_beauty/beauty_3/test_target.pickle","wb") as fr:
    pickle.dump(test_target, fr, protocol=4)    

## Model

In [39]:
from keras.layers import Input, Embedding, LSTM, Dense, Lambda, Multiply
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPool1D
from keras.models import Model
from keras.layers.wrappers import TimeDistributed
import numpy as np
import keras.backend as K
from random import randint

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
description_num = 5
# current_path ='data/'
# local_path = 'data/'

In [40]:
# set parameters:
# V = 142381
# V = 226928
V = len(tokenizer.word_index)
embedding_dim = 50
max_len = maxlen
filters_num = 64
kernel_size = 3

In [43]:
# description_input=Input(shape=(10,50))
# description_validation=Input(shape=(50,))
input_1 = Input(shape=(description_num, max_len))
input_2 = Input(shape=(max_len,))
embedding = Embedding(input_dim=V,
                      output_dim=embedding_dim,
                      input_length=max_len)
conv1d = Conv1D(filters=filters_num,
                kernel_size=kernel_size,
                activation='relu')
maxpool1d = MaxPool1D(maxlen-2)
dense = Dense(15)

# input_vector=TimeDistributed(embedding)(description_input)
# validation_vector=embedding(description_validation)

input_vector = TimeDistributed(embedding)(input_1)
validation_vector = embedding(input_2)

convolutional_vector = TimeDistributed(conv1d)(input_vector)
validation_conv = conv1d(validation_vector)

maxpooling_vector = TimeDistributed(maxpool1d)(convolutional_vector)
validation_maxpooling = maxpool1d(validation_conv)

middle_output = TimeDistributed(dense)(maxpooling_vector)
middle_validation = dense(validation_maxpooling)

In [44]:
print(input_vector.shape, validation_vector.shape,
      convolutional_vector.shape, validation_conv.shape,
      maxpooling_vector.shape, validation_maxpooling.shape,
      middle_output.shape, middle_validation.shape)

(None, 5, 1494, 50) (None, 1494, 50) (None, 5, 1492, 64) (None, 1492, 64) (None, 5, 1, 64) (None, 1, 64) (None, 5, 1, 15) (None, 1, 15)


In [45]:
def change_dim_1(X):
    return K.squeeze(X, 1)

def change_dim_2(X):
    return K.squeeze(X, 2)

def repeat(X):
    return K.repeat_elements(X, description_num, 1)

def repeat1(X):
    return K.repeat_elements(X, 15, 2)

def repeat2(X):
    return K.repeat_elements(X, description_num, 1)

def dot(X, Y):
    return K.dot(X, Y)

def sum_item(X):
    return K.sum(X, axis=2)

def sqrt_item(X):
    return K.sqrt(X)

def cal_denominator(X):
    return 1/(X)

def expand_item(X):
    return K.expand_dims(X, 2)

def expand_rate(X):
    return K.expand_dims(X, 1)

def sum_user(X):
    return K.sum(X, 1)

def sum_rate(X):
    return K.sum(X, 1)

def single_exp(X):
    return K.exp(X)

def sum_exp_denominator(X):
    return 1/(K.sum(K.exp(X), 1))

In [46]:
# def _test_generator(filename):

middle_output_final = Lambda(change_dim_2)(middle_output)#?*10*15
middle_validation_final = Lambda(repeat)(middle_validation)#?*10*15
molecule = Multiply()([middle_output_final, middle_validation_final])
molecule = Lambda(sum_item)(molecule)

denominator1 = Multiply()([middle_output_final, middle_output_final])
denominator1 = Lambda(sum_item)(denominator1)
denominator1 = Lambda(sqrt_item)(denominator1)

denominator2 = Multiply()([middle_validation_final, middle_validation_final])
denominator2 = Lambda(sum_item)(denominator2)
denominator2 = Lambda(sqrt_item)(denominator2)


denominator = Multiply()([denominator1, denominator2])
denominator = Lambda(cal_denominator)(denominator)

similarity = Multiply()([molecule, denominator])
similarity = Lambda(expand_item)(similarity)
similarity = Lambda(repeat1)(similarity)



user = Multiply()([similarity, middle_output_final])
user = Lambda(sum_user)(user)


item = Lambda(change_dim_1)(middle_validation)


rate_hat = Multiply()([user, item])
rate_hat = Lambda(sum_rate)(rate_hat)
rate_hat = Lambda(expand_rate)(rate_hat)

In [47]:
model = Model(inputs=[input_1, input_2], outputs=rate_hat)
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

In [48]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 5, 1494)]    0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 1494)]       0           []                               
                                                                                                  
 time_distributed_4 (TimeDistri  (None, 5, 1494, 50)  1669250    ['input_3[0][0]']                
 buted)                                                                                           
                                                                                                  
 embedding_1 (Embedding)        (None, 1494, 50)     1669250     ['input_4[0][0]']            

In [49]:
hist = model.fit(train_data, train_target,
                 validation_data=(valid_data, valid_target),
                 steps_per_epoch=256,
                 validation_steps=256,
                 epochs=3,
                 verbose=10)

Epoch 1/3


: 

: 

In [ ]:
# scores = model.evaluate_generator(_evaluate_generator('train_whole_list_5s.txt','test_whole_list_5s.txt'),steps=134103)
# scores = model.evaluate_generator(_evaluate_generator('train_whole_list_10.txt','test_whole_list_10.txt'),steps=34266)
# scores = model.evaluate_generator(_evaluate_generator('train_whole_list_6s.txt','test_whole_list_6s.txt'),steps=134103)
test_number = test.shape[0]
scores = model.evaluate(test_data, test_target)
print(scores)